In [66]:
import numpy as np
import matplotlib as plt
from scipy import io, fft, eye
from scipy.io import wavfile

## Input preprocessing

In [75]:
samplingRate, rawData = io.wavfile.read('../audio/xdzibe00.wav')
print('Sampling rate: ' + str(samplingRate) + 'Hz')
print('Audio length:')
print('               ' + str(rawData.size) + ' samples')
print('               ' + str(rawData.size / samplingRate) + ' seconds')
print('Max value:     ' + str(rawData.max()))
print('Min value:     ' + str(rawData.min()))

Sampling rate: 16000Hz
Audio length:
               53556 samples
               3.34725 seconds
Max value:     2635
Min value:     -1516


In [68]:
#normalize the data
normData = rawData / np.linalg.norm(rawData)
#slice into 1024 sample long frames, with 512 sample overlap
slicedData = np.zeros(shape=(int(normData.size / 512),1024))
for i in range(0, int(normData.size / 512) - 1): #the last frame would be incomplete so we ignore it
    slicedData[i] = normData[i * 512:(i * 512) + 1024]


## Fourier transform
In this section we Implement our own version of DFT and compare it with a library implementation.

In [69]:
def customDFT(signal):
    dftMatrix = fft(eye(1024))
    return dftMatrix * signal

In [70]:
#now we can feed the function with our signal

In [82]:
#or just use a library implementation
dftLibrary = np.zeros(shape=(slicedData.shape))
for i in range(0, dftLibrary.shape[0]):
    dftLibrary[i] = np.abs(fft.fft(slicedData[i]))

## Spectrum analysis

## Filter